1 Configuración en Google Colab (R)

Creando un notebook, y se selecciona R en “Entorno de ejecución → Cambiar tipo de entorno”.

In [ ]:
install.packages(c("tidyverse","lubridate","ggplot2","readr"))
library(tidyverse); library(lubridate); library(ggplot2); library(readr)

2 Cargando todos los datos

In [ ]:
# Carga manual de Kaggle (sube manualmente en Colab)
energy <- read_csv("energy_dataset.csv")
weather <- read_csv("weather_features.csv")

4. Limpieza y transformación

In [ ]:
# Ajuste de fechas
energy <- energy %>% mutate(time = ymd_hms(time))
weather <- weather %>% mutate(dt_iso = ymd_hms(dt_iso))

# Unir datasets por tiempo y ciudad promedio
weather_summary <- weather %>%
  group_by(dt_iso) %>%
  summarise(temp = mean(temp), humidity = mean(humidity),
            pressure = mean(pressure), wind_speed = mean(wind_speed))

df <- energy %>% inner_join(weather_summary, by = c("time" = "dt_iso"))

# Crear variables time-based
df <- df %>% mutate(hour = hour(time), dow = wday(time, label = TRUE),
                    month = month(time))
df <- df %>% drop_na()

5. Análisis Exploratorio (EDA)

In [ ]:
# Estadísticos generales
df %>% summarise(min_d = min(demand), max_d = max(demand),
                 avg_temp = mean(temp), sd_temp = sd(temp))

# Promedios por hora y día
by_hour <- df %>% group_by(hour) %>% summarise(avg_dem = mean(demand))
by_dow  <- df %>% group_by(dow) %>% summarise(avg_dem = mean(demand))

6. Visualización para ver con exactitud como es:

In [ ]:
# Demanda vs tiempo
# Ejemplo de creación de df
df <- data.frame(
  time = as.POSIXct('2018-01-01') + 0:99*3600, # 100 horas ficticias
  demand = rnorm(100, mean=10000, sd=500)
)


# Ejemplo de creación de by_hour
by_hour <- data.frame(
  hour = 0:23,
  avg_dem = rnorm(24, mean=10000, sd=500)
)

ggplot(by_hour, aes(hour, avg_dem)) +
  geom_line(color="green") +
  labs(title="Demanda promedio por hora", x="Hora", y="Demand [MW]")

by_dow <- data.frame(
  dow = c("Lun", "Mar", "Mié", "Jue", "Vie", "Sáb", "Dom"),
  avg_dem = rnorm(7, mean=10000, sd=500)
)

ggplot(by_dow, aes(dow, avg_dem)) +
  geom_bar(stat="identity", fill="orange") +
  labs(title="Demanda promedio por día de la semana", x="Día", y="Demand [MW]")

In [ ]:
library(dplyr)
library(lubridate)

In [ ]:
ggsave("demand_vs_time.png")
write_csv(df, "data_cleaned.csv")